<img src="https://drive.google.com/uc?export=view&id=1249lwqHObZj1y3PK_XM8ak5p7UinVV5D" width="100%"></img>

# **Conceptos de SQL Desde Python**
---

En este notebook se da una introducción práctica al lenguaje de consulta SQL desde Python.

Para este taller guiado, debe conectarse a su base de datos `PostgreSQL` como se indica en el notebook `2_postgresql_python.ipynb`.

In [1]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import psycopg2

In [4]:
# Reemplazar parámetros de conexión con los valores de su base de datos:

HOST = "db.bit.io"
DB = "hcquinteroz/mlds3"
USER = "hcquinteroz"
PASSWORD = "v2_3vDhb_Awd5y3bW7eXQHCppgDpJVjp"
PORT = "5432"

En este caso, vamos a ver cómo realizar distintas operaciones CRUD desde SQL:

- **Create**: operaciones de creación de datos (tablas y registros).
- **Read**: operaciones de lectura de datos (consultas).
- **Update**: operaciones de actualización de datos.
- **Delete**: operaciones para eliminar datos (tablas y registros).

Veamos algunas de ellas:

## **1. Creación**
---

* Dentro de una base de datos `sql` por lo general creamos datos en forma tabular (filas y columnas).
* Por organización cada tabla tiene un número definido de campos (columnas) con determinados tipos. SQL estándar define los siguientes tipos de datos:

<img src="https://drive.google.com/uc?export=view&id=1sSM54NkjqSiqVFweM3urGDOgjoAMYehH" width="100%"></img>

### 1.1. Creación de Tablas
---

Veamos la sintaxis general para la creación de una tabla:

```sql
CREATE TABLE <nombre_tabla> (
    <nombre_columna1> <TIPO_COLUMNA> <RESTICCION_COLUMNA>,
    <nombre_columna2> <TIPO_COLUMNA> <RESTICCION_COLUMNA>,
    <nombre_columna3> <TIPO_COLUMNA> <RESTICCION_COLUMNA>,
    PRIMARY KEY <llave_primaria>,
    FOREIGN KEY <llave_foranea> REFERENCES <nombre_otra_tabla> (<columna_otra_tabla>)
);
```

* Cada tabla debe tener una columna que actúe como llave primaria (identificador único).
* Una tabla puede opcionalmente tener llaves foráneas (columnas que se relacionan con columnas de otras tablas).

Veamos cómo crear el siguiente modelo de datos:

<img src="https://drive.google.com/uc?export=view&id=1JUh-yrRlFOrmUbOCccwjxOMNYUJFjs6N" width="20%"></img>

* La instrucción `IF NOT EXISTS` se usa para crear una tabla si esta no ha sido creada antes.
* `SERIAL` es un tipo de datos específico de _PostgreSQL_ que define un número entero que autoincrementa.
* `NOT NULL` es una restricción que evita que existan valores nulos en la columna especificada.

Veamos cómo crear la tabla `estudiantes`:

In [5]:
students_query = """
CREATE TABLE IF NOT EXISTS estudiantes (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INT,
    genero TEXT,
    nota FLOAT
    )
;
"""

Creamos un cursor y ejecutamos la instrucción.

> En éste caso usamos la conexión bajo un contexto `with`, con el fin de cerrar automáticamente la conexión una vez la consulta se ejecuta y evitar problemas de conexión, así mismo, no es necesario ejecutar `conn.close()`.

In [6]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(students_query)
    conn.commit()

Ahora, la tabla `acudientes`:

In [7]:
parents_query = """
CREATE TABLE IF NOT EXISTS acudientes (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INT,
    genero TEXT,
    id_estudiante INT NOT NULL
    )
;
"""

Ejecutamos la instrucción:

In [8]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(parents_query)
    conn.commit()

Puede validar en [bit.io](https://bit.io) que las tablas estén creadas.

### 1.2. Creación de registros (inserción)
---

La sintaxis para crear registros dentro de tablas es la siguiente:

```sql
INSERT INTO
    <nombre_tabla> (<nombre_columna1>, <nombre_columna2>, <nombre_columna3>)
VALUES
    (<valor1>, <valor2>, <valor3>),
    (<valor1>, <valor2>, <valor3>)
;
```

Veamos cómo insertar valores en la tabla `estudiantes`:

In [9]:
data_students = """
INSERT INTO
    estudiantes (nombre, edad, genero, nota)
VALUES
    ('Bart Simpson', 10, 'masculino', 3.0),
    ('Lisa Simpson', 8, 'femenino', 4.9),
    ('Milhouse Van Houten', 11, 'masculino', 2.8),
    ('Ralph Wiggum', 8, 'masculino', 1.0)
    ;
"""

Ejecutamos la instrucción:

In [10]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(data_students)
    conn.commit()

Ahora, vamos a agregar valores a la tabla `acudientes`:

In [11]:
data_parents = """
INSERT INTO
    acudientes (nombre, edad, genero, id_estudiante)
VALUES
    ('Homero Simpson', 39, 'masculino', 1),
    ('Marge Simpson', 33, 'femenino', 1),
    ('Homero Simpson', 39, 'masculino', 2),
    ('Marge Simpson', 33, 'femenino', 2),
    ('Kirk Van Houten', 40, 'masculino', 3),
    ('Luann Van Houten', 32, 'femenino', 3),
    ('Ned Flanders', 59, 'masculino', 5)
    ;
"""

Ejecutamos la instrucción:

In [12]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(data_parents)
    conn.commit()

## **2. Lectura**
---

Ahora, veamos cómo es la sintaxis SQL para la lectura de datos:

```sql
SELECT
    <nombre_columna1> AS <alias1>,
    <OPERACION>(nombre_columna2) AS <alias2>
FROM
    <nombre_tabla> AS <alias_tabla>
JOIN
    <nombre_tabla2> AS <alias_tabla2>
ON
    <alias_tabla>.<alias1> = <alias_tabla2>.<alias1>
WHERE
    <condicion>
GROUP BY
    <nombre_columna1>
HAVING
    <condicion_grupos>
ORDER BY
    <nombre_columna1>
```

Cada una de las palabras reservadas de la consulta tienen la siguiente utilidad:

* `SELECT`: permite especificar qué columnas de la tabla se extraerán.
* `FROM`: indica de qué tabla se extraen los datos.
* `JOIN`: permite cruzar datos de otra tabla, **opcional**.
* `ON`: específica sobre qué columnas se realiza el cruce de tablas, **opcional**.
* `WHERE`: permite filtrar registros de acuerdo a una condición, **opcional**.
* `GROUP BY`: permite agrupar registros de acuerdo a alguna categoría, **opcional**.
* `HAVING`: filtra valores con una condición sobre los grupos, **opcional**.
* `ORDER BY`: ordena los registros según los valores de una columna, **opcional**.

Veamos algunos ejemplos de consultas:

### **2.1. Consultas y Filtros**
---

Veamos cómo extraer todos los registros de una tabla, para ello usamos el operador `*` (indica que vamos a extraer todas las columnas de una tabla):

In [13]:
query = """
SELECT
    *
FROM
    acudientes
;
"""

In [14]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[(1, 'Homero Simpson', 39, 'masculino', 1), (2, 'Marge Simpson', 33, 'femenino', 1), (3, 'Homero Simpson', 39, 'masculino', 2), (4, 'Marge Simpson', 33, 'femenino', 2), (5, 'Kirk Van Houten', 40, 'masculino', 3), (6, 'Luann Van Houten', 32, 'femenino', 3), (7, 'Ned Flanders', 59, 'masculino', 5)]


El método `fetchall` permite extraer los resultados de una consulta.

Adicionalmente, el resultado obtenido es una lista de tuplas donde cada elemento tiene los tipos de datos que generalmente se manejan en _Python_ (`int`, `str`, `float`, entre otros).

In [15]:
print(type(data))

<class 'list'>


In [16]:
print(type(data[0]))

<class 'tuple'>


In [17]:
print(type(data[0][0]))

<class 'int'>


Ahora, veamos un ejemplo de selección condicional. Vamos a extraer los nombres y las edades de los acudientes que son de género masculino:

In [20]:
query = """
SELECT
    nombre, edad
FROM
    acudientes
WHERE
    genero = 'masculino'
;
"""

In [21]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Homero Simpson', 39), ('Homero Simpson', 39), ('Kirk Van Houten', 40), ('Ned Flanders', 59)]


También podemos anidar condiciones usando operadores lógicos, por ejemplo, para seleccionar todos los acudientes de género masculino y con menos de 50 años de edad:

In [24]:
query = """
SELECT
    nombre, edad
FROM
    acudientes
WHERE
    genero = 'masculino' AND
    edad < 50
"""

In [25]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Homero Simpson', 39), ('Homero Simpson', 39), ('Kirk Van Houten', 40)]


Estos son los operadores en SQL estandar:

#### **2.1.1. Aritméticos**

| Operador | Descripción | Ejemplo |
| --- | --- | --- |
| `+` | Suma valores | `columna1 + columna2` |
| `-` | Resta valores | `columna1 - columna2` |
| `*` | Multiplica valores | `columna1 * 2` |
| `/` | Divide valores | `columna1 / columna2` |
| `%` | Módulo de dos valores | `columna1 % 2` |


#### **2.1.2. Comparación**

| Operador | Descripción | Ejemplo |
| --- | --- | --- |
| `=` | Igualdad | `columna1 = 'caso'` |
| `>` | Mayor que | `columna1 > columna2` |
| `<` | Menor que | `columna1 < columna2` |
| `>=` | Mayor o igual que | `columna1 >= columna2` |
| `<=` | Menor o igual que | `columna1 <= columna2` |
| `<>` | Diferente | `columna1 <> 7` |


#### **2.1.3. Operadores Lógicos**

| Operador | Descripción | Ejemplo |
| --- | --- | --- |
| `AND` | Operación and | `columna1 AND columna2` |
| `OR` | Operación or | `columna1 OR columna2` |
| `NOT` | Negación | `NOT columna1` |
| `ALL` | Verdadero si todos los valores cumplen | `ALL columna1` |
| `ANY` | Verdadero si al menos un valor cumple | `ANY columna1` |
| `BETWEEN` | Rango de valores | `columna1 BETWEEN 1 AND 2` |
| `EXISTS` | Verdadero si al menos hay un valor no nulo | `EXISTS columna1` |
| `IN` | Pertenencia | `columna1 IN ('caso1', 'caso2')` |
| `LIKE` | Verdadero si hay coincidencia con un patrón | `column1 LIKE 'Marc%'` |


#### **2.1.4. Ejemplo de consulta**
Por ejemplo, podemos seleccionar el doble de la edad de los acudientes que comiencen por `Homero`:

In [26]:
query = """
SELECT
    nombre,
    edad * 2 AS doble_edad
FROM
    acudientes
WHERE
    nombre LIKE 'Homero%'
;
"""

In [27]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Homero Simpson', 78), ('Homero Simpson', 78)]


> Puede ver más información sobre los patrones `LIKE` en el siguiente enlace[Patrones LIKE](https://www.w3schools.com/sql/sql_like.asp).

### **2.2. Cruce de Tablas**
---

En SQL existen diversas formas de cruzar dos tablas, más precisamente, se suelen manejar los siguientes tipos de `JOIN`:

<img src="https://drive.google.com/uc?export=view&id=1iJQyu75yJEqFG0a5iHDK4UaBKukgZKOD" width="60%"></img>

* `INNER JOIN`: selecciona unicamente los registros que se encuentran en ambas tablas (intersección).
* `OUTER JOIN`: selecciona todos los registros de ambas tablas (unión).
* `LEFT JOIN`: selecciona los todos los registros de la primer tabla (así no se encunetren en la segunda).
* `RIGHT JOIN`: selecciona los todos los registros de la segunda tabla (así no se encunetren en la primera).

Veamos estos join con los datos de estudiantes:

Primero, el `INNER JOIN` debe seleccionar unicamente los estudiantes y acudientes que corresponden según la columna `id_estudiante`.

In [28]:
query = """
SELECT
    l.nombre AS nombre_estudiante,
    r.nombre AS nombre_acudiente
FROM
    estudiantes AS l
INNER JOIN
    acudientes AS r
ON
    l.id = r.id_estudiante
;
"""

In [29]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Bart Simpson', 'Homero Simpson'), ('Bart Simpson', 'Marge Simpson'), ('Lisa Simpson', 'Homero Simpson'), ('Lisa Simpson', 'Marge Simpson'), ('Milhouse Van Houten', 'Kirk Van Houten'), ('Milhouse Van Houten', 'Luann Van Houten')]


Segundo, el `FULL OUTER JOIN` debe seleccionar todos los estudiantes y acudientes sin importar si no están relacionados:

In [30]:
query = """
SELECT
    l.nombre AS nombre_estudiante,
    r.nombre AS nombre_acudiente
FROM
    estudiantes AS l
FULL OUTER JOIN
    acudientes AS r
ON
    l.id = r.id_estudiante
;
"""

In [31]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Bart Simpson', 'Homero Simpson'), ('Bart Simpson', 'Marge Simpson'), ('Lisa Simpson', 'Homero Simpson'), ('Lisa Simpson', 'Marge Simpson'), ('Milhouse Van Houten', 'Kirk Van Houten'), ('Milhouse Van Houten', 'Luann Van Houten'), (None, 'Ned Flanders'), ('Ralph Wiggum', None)]


Tercero, el `LEFT JOIN` debe seleccionar todos los estudiantes, los acudientes solo se recuperan si están relacionados a un estudiante:

In [32]:
query = """
SELECT
    l.nombre AS nombre_estudiante,
    r.nombre AS nombre_acudiente
FROM
    estudiantes AS l
LEFT JOIN
    acudientes AS r
ON
    l.id = r.id_estudiante
;
"""

In [33]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Bart Simpson', 'Homero Simpson'), ('Bart Simpson', 'Marge Simpson'), ('Lisa Simpson', 'Homero Simpson'), ('Lisa Simpson', 'Marge Simpson'), ('Milhouse Van Houten', 'Kirk Van Houten'), ('Milhouse Van Houten', 'Luann Van Houten'), ('Ralph Wiggum', None)]


Por último, el `RIGHT JOIN` debe seleccionar todos los acudientes, los estudiantes solo se recuperan si están relacionados a un acudiente:

In [34]:
query = """
SELECT
    l.nombre AS nombre_estudiante,
    r.nombre AS nombre_acudiente
FROM
    estudiantes AS l
RIGHT JOIN
    acudientes AS r
ON
    l.id = r.id_estudiante
;
"""

In [35]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Bart Simpson', 'Homero Simpson'), ('Bart Simpson', 'Marge Simpson'), ('Lisa Simpson', 'Homero Simpson'), ('Lisa Simpson', 'Marge Simpson'), ('Milhouse Van Houten', 'Kirk Van Houten'), ('Milhouse Van Houten', 'Luann Van Houten'), (None, 'Ned Flanders')]


Adicional a lo anterior, se puede realizar cualquier operación de `JOIN` sobre varias columnas al tiempo, anidándolas con el operador `AND`:

```sql
...
INNER JOIN
    l.columna1 = r.columna2 AND
    l.columna2 = r.columna2
...
```

### **2.3. Agrupamiento y Agregación**
---

Las operaciones de agrupamiento y agregación consisten en el uso de `GROUP BY` en conjunto con operaciones de agregación como:

| Operación | Descripción |
| --- | --- |
| `SUM` | suma |
| `COUNT` | recuento |
| `AVG` | promedio |
| `MIN` | mínimo |
| `MAX` | máximo |

De forma general, las operaciones de agrupamiento y agregación buscan hacer dos cosas:

1. Agrupar registros de acuerdo a una o varias columnas categóricas.
2. Reducir columnas de cada grupo usando una función de agregación.

Veamos un ejemplo gráfico:

<img src="https://drive.google.com/uc?export=view&id=1_j8hNDzAJ69JxyuJCIXP4fwWWO9UT62Z"></img>

Desde SQL:

In [36]:
query = """
SELECT
    genero,
    AVG(nota) AS nota_promedio
FROM
    estudiantes
GROUP BY
    genero
;
"""

In [37]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('masculino', 2.2666666666666666), ('femenino', 4.9)]


También podemos crear categorías a partir de variables numéricas .

Con `CASE` puede crear nuevas variables de acuerdo a distintas condiciones acotadas por `WHEN`.

In [38]:
query = """
SELECT
    nombre,
    CASE
        WHEN edad < 10 THEN 'menor'
        WHEN edad >= 10 THEN 'mayor'
    END AS menor_mayor
FROM
    estudiantes
;
"""

In [39]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('Bart Simpson', 'mayor'), ('Lisa Simpson', 'menor'), ('Milhouse Van Houten', 'mayor'), ('Ralph Wiggum', 'menor')]


Veamos el uso con `GROUP BY`, para calcular el promedio de notas por genero y por rango de edad:

In [40]:
query = """
SELECT
    genero,
    CASE
        WHEN edad < 10 THEN 'menor'
        WHEN edad >= 10 THEN 'mayor'
    END AS menor_mayor,
    AVG(nota)
FROM
    estudiantes
GROUP BY
    genero, menor_mayor
;
"""

In [41]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[('masculino', 'mayor', 2.9), ('masculino', 'menor', 1.0), ('femenino', 'menor', 4.9)]


##  **3. Actualización**
---

Para actualizar valores dentro de una tabla debemos usar la instrucción `UPDATE`, la cual tiene la siguiente sintaxis:

```sql
UPDATE
    <nombre_tabla>
SET
    <nombre_columna1> = <valor1>,
    <nombre_columna2> = <valor2>
WHERE
    <condicion>
;
```

Veamos un ejemplo de actualización

In [42]:
update_query = """
UPDATE
    acudientes
SET
    edad = 40
WHERE
    nombre LIKE 'Homero%'
;
"""

In [43]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(update_query)
    conn.commit()

Podemos validar el resultado:

In [44]:
validation_query = """
SELECT
    edad
FROM
    acudientes
WHERE
    nombre LIKE 'Homero%'
;
"""

In [45]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(validation_query)
    data = cursor.fetchall()
print(data)

[(40,), (40,)]


Dentro de los métodos de actualización, también tenemos comandos `ALTER`, los cuales permiten modificar columnas de una tabla existente, la sintaxis es la siguiente:

```sql
ALTER TABLE
    <nombre_tabla>
<OPERACION>
    <nombre_columna> <TIPO_COLUMNA>
;
```

Veamos como crear una columna en la tabla de estudiantes.

In [46]:
update_query = """
ALTER TABLE
    estudiantes
ADD
    bonos FLOAT
;
"""

In [47]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(update_query)
    conn.commit()

Puede usar la siguiente consulta para ver el esquema de la tabla `estudiantes`:

In [48]:
schema_query = """
SELECT 
   table_name, 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'estudiantes'
;
"""

In [49]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(schema_query)
    data = cursor.fetchall()
print(data)

[('estudiantes', 'id', 'integer'), ('estudiantes', 'edad', 'integer'), ('estudiantes', 'nota', 'double precision'), ('estudiantes', 'bonos', 'double precision'), ('estudiantes', 'nombre', 'text'), ('estudiantes', 'genero', 'text')]


También podemos modificar tipos en columnas ya existentes:

In [50]:
update_query = """
ALTER TABLE
    estudiantes
ALTER COLUMN
    bonos
TYPE
    INT
;
"""

In [51]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(update_query)
    conn.commit()

Validemos el esquema de la tabla:

In [52]:
schema_query = """
SELECT 
   table_name, 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'estudiantes'
;
"""

In [53]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(schema_query)
    data = cursor.fetchall()
print(data)

[('estudiantes', 'id', 'integer'), ('estudiantes', 'edad', 'integer'), ('estudiantes', 'nota', 'double precision'), ('estudiantes', 'bonos', 'integer'), ('estudiantes', 'nombre', 'text'), ('estudiantes', 'genero', 'text')]


## **4. Borrado**
---

Para eliminar registros debemos usar la sintaxis de `DELETE`:

```sql
DELETE FROM
    <nombre_tabla>
WHERE
    <condicion>
;
```

Veamos un ejemplo

In [54]:
delete_query = """
DELETE FROM
    acudientes
WHERE
    nombre LIKE 'Homero%'
;
"""

In [55]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(delete_query)
    conn.commit()

Ahora, podemos validar la tabla:

In [56]:
query = """
SELECT
    *
FROM
    acudientes
;
"""

In [57]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
print(data)

[(2, 'Marge Simpson', 33, 'femenino', 1), (4, 'Marge Simpson', 33, 'femenino', 2), (5, 'Kirk Van Houten', 40, 'masculino', 3), (6, 'Luann Van Houten', 32, 'femenino', 3), (7, 'Ned Flanders', 59, 'masculino', 5)]


También podemos eliminar columnas con comandos `ALTER`

In [58]:
alter_query = """
ALTER TABLE
    acudientes
DROP
    genero
;
"""

In [59]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

Veamos el esquema:

In [60]:
schema_query = """
SELECT 
   table_name, 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'acudientes'
;
"""

In [61]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(schema_query)
    data = cursor.fetchall()
print(data)

[('acudientes', 'id', 'integer'), ('acudientes', 'edad', 'integer'), ('acudientes', 'id_estudiante', 'integer'), ('acudientes', 'nombre', 'text')]


Finalmente, si deseamos eliminar tablas debemos usar sentencias `DROP`. Con la siguiente sintaxis:

```sql
DROP TABLE
    <nombre_tabla>
;
```

Veamos un ejemplo:

In [62]:
drop_query = """
DROP TABLE estudiantes;
DROP TABLE acudientes;
"""

In [63]:
with psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD,
        port=PORT
        ) as conn:
    cursor = conn.cursor()
    cursor.execute(drop_query)
    conn.commit()

Puede validar en [bit.io](https://bit.io) que ya no existe ninguna tabla.

## **5. Recursos Adicionales**
---

* [Khan Academy - Introducción a SQL: consulta y gestión de datos](https://es.khanacademy.org/computing/computer-programming/sql)
* [Kaggle - Intro to SQL](https://www.kaggle.com/learn/intro-to-sql)
* [Kaggle - Advanced SQL](https://www.kaggle.com/learn/advanced-sql)
* [Coursera - Introduction to Structured Query Language (SQL) - University of Michigan](https://www.coursera.org/learn/intro-sql)
* [Udacity - Learn SQL Nanodegree Program](https://www.udacity.com/course/learn-sql--nd072)

## **6. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*